In [1]:
import keras
from keras import layers
import numpy as np
import random
import io


In [2]:
path = "text.txt"
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # Eliminar las nuevas lineas
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cortar el texto en bloques iguales
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 2188
Total chars: 43
Number of sequences: 716


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [4]:
def sample(preds, temperature=1.0):
    # Funcion que permite muestrear 
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [5]:
epochs = 100
batch_size = 32

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("------- GENERADO DESPUÉS DE LA ÉPOCA: %d -------------------------------------------------------------------------------------------------------------------------------------------------" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("\tDIVERSIDAD: ", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('\t   Generando con semilla: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("\t   Generado --> ", generated)
        print("\t--------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.3621

------- GENERADO DESPUÉS DE LA ÉPOCA: 0 -------------------------------------------------------------------------------------------------------------------------------------------------
	DIVERSIDAD:  0.2
	   Generando con semilla: " 1998 cuando el entonces gobernador cons"
	   Generado -->  rrrrerr rrrrr crre crcrriercrrrrr rr   rrrrrrr crrr cr rr rcrcr crcrrrrrrrrc rr c irc rerrrr re c cr  rrrrr rrrrc crr crrr r ri rr rrerrr ercrr rrrcrerc rrrr crrrrrsrerircrrerr rccrcrrrrerrrr  r e rrrr rr rs crcrrrrcr r r ecrrrr r rr rr ccrrrrrccrr rr rcrrrrrscrccrr cceie  rrrr r r crrrrr rcrrcrrrc rrrr rr rcrrrrerrr tr rrr rsr  rc rercr rrrr rrrrcrrrrrr  rrrrrrrrcrrr rcrrrrrr rrrrrc r  ccrrrcrrrr
	--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
	DIVERSIDAD:  0.5
	   Generando con semilla: " 1998 cuando el entonces gobernado

In [6]:
def generate_text(seed_text, length=400, temperature=1.0):
    generated = ""
    sentence = seed_text[-maxlen:]  # Asegúrate de que la semilla tenga longitud `maxlen`
    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            if char in char_indices:
                x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
    return generated

In [9]:
# Ejemplo de uso
seed_text = "Nuestra misión es formar profesionistas competentes y"
print(generate_text(seed_text, length=100, temperature=0.5))

 púriniería ingunicría amtuvidades demmpirras s púemion la ingeniería en enticiempra en cimpureces d
